In [86]:
from openai import OpenAI

def chat_with_gpt(user_prompt, system_prompt, model='gpt-4o-mini', temperature = 0.1):
    
    client = OpenAI()
    
    completion = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    )

    return completion.choices[0].message.content

In [51]:
def transform_patient_data(raw_data):
    transformed_data = []
    for entry in raw_data:
        lines = entry.split("\n")
        patient_info = {
            "Age": lines[0].split(": ")[1].strip(),
            "Gender": lines[1].split(": ")[1].strip(),
            "Existing Medical Conditions": lines[2].split(": ")[1].strip(),
            "Symptoms": lines[3].split(": ")[1].strip(),
            "Key Past Experiences": [
                line.strip("- ").strip()
                for line in lines[4].split(": ", 1)[1].strip().split("\n")
                if line.strip()
            ],
        }
        transformed_data.append(patient_info)
    return transformed_data

In [34]:
import json
import re

# JSON 파일 경로
file_path = 'C:\\Users\\dmkim\\OneDrive\\바탕 화면\\엔피시즈\\code\\original_persona_data.json'

# JSON 파일 열기
with open(file_path, 'r', encoding='utf-8') as file:
    prompt_json = json.load(file)

# JSON 데이터 출력
for i in prompt_json:
    print(i['content'])

Patient A was a 54-year-old nun who described herself as ‘detail-oriented and perfectionistic.’ She was diagnosed with breast cancer 5 years ago when she visited the hospital for a routine examination. Along with breast cancer, she was also diagnosed with depressed mood and insomnia. When she visited about 6 months ago, her back pain became worse, which subsequently made it more difficult for her to sleep. She initiated treatment in pain control and physical therapy in the Department of Rehabilitation medicine as an outpatient. She was prescribed 10 mg of Zolpidem. Her pain and insomnia improved, but she continued to take sleep medication on an as needed basis. Her main goal for participating in this treatment was discontinuation of her sleep medication.
Patient B was a 52-year-old public official who had timid and sensitive personality. He started to experience anxiety and heart palpitations, physical symptoms of dizziness and sleep disorders about two years ago when his mother passed

In [124]:
system_prompt = """

You are a psychological expert. From the given content, extract and summarize the following information about the patient:

Age:
Gender:
Existing medical conditions:
Symptoms:
Key past experience: [A,B,C,D,E] 

For experiences A~E, focus on issues the patient has faced and express experiences concisely and from the patient's perspective.
Experiences must be tied up into a list format with only experience. This is example format [I've ate pizza for lunch, I went to the restroom, and so on]
Age should contain only the number
Do not change provided format, it should be exatly same as provided include spacing
"""

In [125]:
patient_dataset = []
for patient in prompt_json:
    data = chat_with_gpt(patient["content"], system_prompt, model='gpt-4o')
    patient_dataset.append(data)
    
print(patient_dataset)

['Age: 54  \nGender: Female  \nExisting medical conditions: Breast cancer, depressed mood, insomnia, back pain  \nSymptoms: Difficulties sleeping, worsened back pain  \n\nKey past experience: [I was diagnosed with breast cancer during a routine examination, I experienced a depressed mood and insomnia, my back pain got worse impacting my sleep, I started pain control and physical therapy, I wanted to stop taking sleep medication]', 'Age: 52  \nGender: Male  \nExisting medical conditions: Sleep disorders  \nSymptoms: Anxiety, heart palpitations, dizziness  \nKey past experience: [My mother passed away, I started experiencing anxiety and heart palpitations, I took sleep medication with no improvement, I participated in cognitive-behavioral therapy for insomnia, the therapy was not very effective]  ', "Age: 64  \nGender: Female  \nExisting medical conditions: Anxiety  \nSymptoms: Insomnia  \nKey past experience: [Conflicts with daughter-in-law started six years ago, Insomnia started after 

In [126]:
json_list = []

for patient in patient_dataset:
    patient = [item for item in patient.split('\n') if item!= ""]
    # print(patient)
    age = patient[0].split(': ')[1]
    gender = patient[1].split(': ')[1]
    condition = patient[2].split(': ')[1]
    symptoms = patient[3].split(': ')[1]
    experience_string = patient[4].split(': ')[1]
    experience = [item.strip() for item in experience_string.strip("[]").split(",")]
    # print(age)
    # print(gender)
    # print(condition)
    # print(symptoms)
    # print(experience)

    json_patient = {
        "Age": age,
        "Gender": gender,
        "Existing Medical Conditions": condition,
        "Symptoms": symptoms,
        "Experience": experience 
    }
    json_list.append(json_patient)

file_path = 'processed_persona_data_2ver.json'
with open(file_path, 'w', encoding='utf-8') as file:
    json.dump(json_list, file, ensure_ascii=False, indent=4)

print(f"Data saved to {file_path}")


Data saved to processed_persona_data_2ver.json
